In [2]:
import numpy as np 
import pandas as pd 
import os
import matplotlib.pyplot as plt
import matplotlib.image as mplimg
from matplotlib.pyplot import imshow

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

from keras import layers
from keras.preprocessing import image
from keras.applications.imagenet_utils import preprocess_input
from keras.layers import Input, Dense, Activation, BatchNormalization, Flatten, Conv2D
from keras.layers import AveragePooling2D, MaxPooling2D, Dropout
from keras.models import Model

import keras.backend as K
from keras.models import Sequential

import warnings
warnings.simplefilter("ignore", category=DeprecationWarning)

C:\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [4]:
os.listdir("./dataset/")

['sample_submission.csv',
 'submission.csv',
 'submission2.csv',
 'test',
 'train',
 'train.csv',
 'train_classes']

# Preparing Training Data

In [25]:
train_df = pd.read_csv("./dataset/train.csv")
train_df.head()

,Image,Id
0,0000e88ab.jpg,w_f48451c
1,0001f9222.jpg,w_c3d896a
2,00029d126.jpg,w_20df2c5
3,00050a15a.jpg,new_whale
4,0005c1ef8.jpg,new_whale


In [26]:
train_df.shape

(25361, 2)

In [9]:
train_df.loc[train_df.Id=='new_whale'].count()

Image    9664
Id       9664
dtype: int64

In [20]:
def prepareImages(data, m, dataset):
    print("Preparing images")
    X_train = np.zeros((m, 100, 100, 3))
    count = 0
    
    for fig in data['Image']:
        #load images into images of size 100x100x3
        img = image.load_img("./dataset/"+dataset+"/"+fig, target_size=(100, 100, 3))
        x = image.img_to_array(img)
        x = preprocess_input(x)
        print(image)

        X_train[count] = x
        if (count%500 == 0):
            print("Processing image: ", count+1, ", ", fig)
        count += 1
    
    return X_train

In [21]:
def prepare_labels(y):
    values = np.array(y)
    label_encoder = LabelEncoder()
    integer_encoded = label_encoder.fit_transform(values)
    # print(integer_encoded)

    onehot_encoder = OneHotEncoder(sparse=False)
    integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
    onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
    # print(onehot_encoded)

    y = onehot_encoded
    # print(y.shape)
    return y, label_encoder

In [23]:
X = prepareImages(train_df[:1], 1, "train")
X /= 255

Preparing images


FileNotFoundError: [Errno 2] No such file or directory: './dataset/test/0000e88ab.jpg'

In [24]:
X = prepareImages(train_df, train_df.shape[0], "train")
X /= 255

Preparing images


FileNotFoundError: [Errno 2] No such file or directory: './dataset/train/0000e88ab.jpg'

In [7]:
y, label_encoder = prepare_labels(train_df['Id'])

In [1]:
y

NameError: name 'y' is not defined

In [8]:
y.shape


(25361, 5005)

In [12]:
b = np.arange(6)
b[1] = 5
print(b)

print(np.argmax(b))

[0 5 2 3 4 5]
1


# Quick CNN network

In [10]:
model = Sequential()

model.add(Conv2D(32, (7, 7), strides = (1, 1), name = 'conv0', input_shape = (100, 100, 3)))

#model.add(BatchNormalization(axis = 3, name = 'bn0'))
model.add(Activation('relu'))

model.add(MaxPooling2D((3, 3), name='max_pool'))
model.add(Conv2D(64, (5, 5), strides = (1,1), name="conv1"))
model.add(Activation('relu'))
model.add(AveragePooling2D((3, 3), name='avg_pool'))

model.add(Flatten())
model.add(Dense(500, activation="relu", name='rl'))
model.add(Dropout(0.8))
model.add(Dense(y.shape[1], activation='softmax', name='sm'))

model.compile(loss='categorical_crossentropy', optimizer="adam", metrics=['accuracy'],)
model.summary()

TypeError: softmax() got an unexpected keyword argument 'axis'

In [ ]:
VALIDATION_SPLIT = 0.2
history = model.fit(X, y, epochs=30, batch_size=32, verbose=1, validation_split=VALIDATION_SPLIT)
#gc.collect()

# Testing and evaluating

In [35]:
test = os.listdir("./data/test/")
print(len(test))

7960


In [36]:
col = ['Image']
test_df = pd.DataFrame(test, columns=col)
test_df['Id'] = ''

In [38]:
model.save_weights('./second_try_27_12.h5')

In [37]:
A = prepareImages(test_df, test_df.shape[0], "test")
A /= 255

Preparing images
Processing image:  1 ,  00028a005.jpg
Processing image:  501 ,  0f9219c40.jpg
Processing image:  1001 ,  1ea95c247.jpg
Processing image:  1501 ,  2fe4ec2d5.jpg
Processing image:  2001 ,  404206f77.jpg
Processing image:  2501 ,  4fc69fd82.jpg
Processing image:  3001 ,  5ffc2d300.jpg
Processing image:  3501 ,  7042be267.jpg
Processing image:  4001 ,  801c6a722.jpg
Processing image:  4501 ,  906aa36c8.jpg
Processing image:  5001 ,  a0d337cdc.jpg
Processing image:  5501 ,  b1a4e4c18.jpg
Processing image:  6001 ,  c0b378682.jpg
Processing image:  6501 ,  cfe9d9d04.jpg
Processing image:  7001 ,  e00620fc3.jpg
Processing image:  7501 ,  f1b504951.jpg


In [39]:
predictions = model.predict(np.array(A))


In [40]:
for i, pred in enumerate(predictions):
    test_df.loc[i, 'Id'] = ' '.join(label_encoder.inverse_transform(pred.argsort()[-5:][::-1]))

In [41]:
test_df.head(10)
test_df.to_csv('./data/submission2.csv', index=False)